In [1]:
from pandas import read_csv as read
import pandas as pd
import numpy as np
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt
from sklearn import metrics
from plot import pltPRcurve, pltROCcurve 

In [2]:
def modExec():
    model.fit(X_train, y_train)
    expected = y_test
    predicted = model.predict(X_test)

    pobas = model.predict_proba(X_test)
    preds = pobas[:, 1]

    report = metrics.classification_report(expected, predicted)
    conf_matrix = metrics.confusion_matrix(expected, predicted)

    print(type(model).__name__)
    print(report)
    print(conf_matrix)
    ##print(metrics.roc_auc_score(expected, preds))

## Данные до исследования признаков

In [3]:
path = "filtData.csv"
data = read(path, delimiter=",")

data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)
data.drop('oldbalanceDest', axis=1, inplace=True)
data.drop('step', axis=1, inplace=True)
data.drop('newbalanceOrig', axis=1, inplace=True)
data.drop('newbalanceDest', axis=1, inplace=True)
'''
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['type'] = le.fit_transform(data['type'])
'''

data = pd.get_dummies(data)
data.head()

,amount,oldbalanceOrg,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,9839.64,170136.00,0,0,0,0,1,0
1,181.00,181.00,1,0,0,0,0,1
2,181.00,181.00,1,0,1,0,0,0
3,9478.39,116494.00,0,0,0,0,1,0
4,3454.08,9031.96,0,0,0,0,1,0


In [4]:
data.corr()

,amount,oldbalanceOrg,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
amount,1.000000,0.205667,0.346157,-0.054279,0.023805,-0.021563,-0.181568,0.304017
oldbalanceOrg,0.205667,1.000000,0.070277,0.464094,-0.180977,-0.022039,-0.190083,-0.034110
isFraud,0.346157,0.070277,1.000000,-0.142891,0.082316,-0.021918,-0.189924,0.340712
type_CASH_IN,-0.054279,0.464094,-0.142891,1.000000,-0.382152,-0.039765,-0.344581,-0.182274
type_CASH_OUT,0.023805,-0.180977,0.082316,-0.382152,1.000000,-0.058617,-0.507939,-0.268685
type_DEBIT,-0.021563,-0.022039,-0.021918,-0.039765,-0.058617,1.000000,-0.052854,-0.027958
type_PAYMENT,-0.181568,-0.190083,-0.189924,-0.344581,-0.507939,-0.052854,1.000000,-0.242270
type_TRANSFER,0.304017,-0.034110,0.340712,-0.182274,-0.268685,-0.027958,-0.242270,1.000000


In [5]:
X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()


RandomForestClassifier
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23790
           1       0.98      0.98      0.98      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.99      0.99     28124
weighted avg       0.99      0.99      0.99     28124

[[23717    73]
 [   83  4251]]


## Данные после исследования признаков

In [6]:
import numpy as np
from pandas import read_csv as read
import pandas as pd
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

path = "newFilt.csv"
data = read(path, delimiter=",")

data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)
data.drop('oldbalanceDest', axis=1, inplace=True)
data.drop('step', axis=1, inplace=True)
data.drop('newbalanceOrig', axis=1, inplace=True)
data.drop('newbalanceDest', axis=1, inplace=True)

data = pd.get_dummies(data)
data.head()

,amount,oldbalanceOrg,isFraud,hour,newSender,newReceiver,merchant,fraudsEarly,LTS,LTR,IZoB,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,9839.64,170136.00,0,1,1,1,1,0,0,0,0,0,0,0,1,0
1,181.00,181.00,1,1,1,1,0,0,0,0,1,0,0,0,0,1
2,181.00,181.00,1,1,1,1,0,0,0,0,1,0,1,0,0,0
3,9478.39,116494.00,0,1,1,1,1,0,0,0,0,0,0,0,1,0
4,3454.08,9031.96,0,1,1,1,1,0,0,0,0,0,0,0,1,0


In [7]:
data.corr()

,amount,oldbalanceOrg,isFraud,hour,newSender,newReceiver,merchant,fraudsEarly,LTS,LTR,IZoB,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
amount,1.000000,0.205667,0.346157,-0.081528,0.001350,-0.026341,-0.181568,0.002111,-0.001002,0.033480,0.119121,-0.054279,0.023805,-0.021563,-0.181568,0.304017
oldbalanceOrg,0.205667,1.000000,0.070277,-0.012752,0.001086,-0.053759,-0.190083,0.008077,-0.001170,0.036172,-0.299378,0.464094,-0.180977,-0.022039,-0.190083,-0.034110
isFraud,0.346157,0.070277,1.000000,-0.212034,0.001871,0.035525,-0.189924,-0.000383,-0.001478,-0.002724,0.220020,-0.142891,0.082316,-0.021918,-0.189924,0.340712
hour,-0.081528,-0.012752,-0.212034,1.000000,-0.003358,-0.007349,0.053131,-0.002751,0.001089,0.002095,-0.062185,0.022327,-0.019689,-0.004976,0.053131,-0.075162
newSender,0.001350,0.001086,0.001871,-0.003358,1.000000,0.003078,0.004512,0.000494,-0.789905,-0.001643,0.005377,-0.003201,-0.000551,0.000521,0.004512,-0.001815
newReceiver,-0.026341,-0.053759,0.035525,-0.007349,0.003078,1.000000,0.195419,-0.256098,-0.000518,-0.728641,-0.018265,-0.079805,-0.101021,-0.008383,0.195419,-0.029270
merchant,-0.181568,-0.190083,-0.189924,0.053131,0.004512,0.195419,1.000000,-0.050135,-0.003564,-0.142390,-0.113351,-0.344581,-0.507939,-0.052854,1.000000,-0.242270
fraudsEarly,0.002111,0.008077,-0.000383,-0.002751,0.000494,-0.256098,-0.050135,1.000000,-0.000390,0.193658,0.005702,0.017185,0.032106,0.001986,-0.050135,0.002377
LTS,-0.001002,-0.001170,-0.001478,0.001089,-0.789905,-0.000518,-0.003564,-0.000390,1.000000,0.000133,-0.002108,0.003712,0.001559,-0.000411,-0.003564,-0.001775
LTR,0.033480,0.036172,-0.002724,0.002095,-0.001643,-0.728641,-0.142390,0.193658,0.000133,1.000000,0.019275,0.052543,0.076151,0.010475,-0.142390,0.023555


In [8]:
X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

from sklearn import metrics

### Рандомный лес

In [9]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=110, n_jobs=-1)

modExec()

RandomForestClassifier
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     23790
           1       0.99      0.95      0.97      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.98      0.98     28124
weighted avg       0.99      0.99      0.99     28124

[[23735    55]
 [  202  4132]]


### Решающее дерево

In [10]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

modExec()

DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     23790
           1       0.99      0.97      0.98      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.98      0.99     28124
weighted avg       0.99      0.99      0.99     28124

[[23739    51]
 [  139  4195]]


### К соседей

In [11]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=20)

modExec()

KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     23790
           1       0.96      0.96      0.96      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.98      0.97      0.98     28124
weighted avg       0.99      0.99      0.99     28124

[[23621   169]
 [  190  4144]]


### ГаусианНБ

In [12]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

modExec()

GaussianNB
              precision    recall  f1-score   support

           0       0.87      0.99      0.93     23790
           1       0.86      0.19      0.32      4334

   micro avg       0.87      0.87      0.87     28124
   macro avg       0.86      0.59      0.62     28124
weighted avg       0.87      0.87      0.83     28124

[[23652   138]
 [ 3497   837]]


### Градиентный  бустинг

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(max_depth=4)

modExec()

GradientBoostingClassifier
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     23790
           1       0.99      0.93      0.96      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.96      0.97     28124
weighted avg       0.99      0.99      0.99     28124

[[23737    53]
 [  307  4027]]
